In [1]:
!pip install -q sentence-transformers wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.2 MB/s eta 0:00:00:00:0100:01


In [2]:
# Cell 1: Seed và imports
import os, random, math
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sentence_transformers import SentenceTransformer
import wandb

# --- Seed để tái lập ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device={device}, torch={torch.__version__}")

# W&B init
Name = "NgocMinh"
Model_name = "LightFM"
Version = "1.0.0"

wandb.login(key="62e3cf4c2815c959ed2609de1d55fa0504818c4a")

# 1.1. Khởi tạo W&B run
wandb.init(
    project="hybrid-neumf-llm",
    name="HybridNeuMF_withLLM",
    config={
        "mf_dim": 32,
        "mlp_layers": [64, 32, 16, 8],
        "llm_model": "all-MiniLM-L6-v2",  # 🟢 CÓ dòng này
        "llm_dim": 384,
        "batch_size": 1024,
        "num_neg": 4,
        "lr": 1e-3,
        "weight_decay": 1e-5,
        "epochs": 10,
        "K": 10
    }
)
cfg = wandb.config



Using device=cuda, torch=2.6.0+cu124


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kieusontung8 (kieusontung8-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
# Cell 2: Load ratings, movies, users
ratings = pd.read_csv("/kaggle/input/nicedataset/ratings.csv")    # user_id,item_id,rating,timestamp
movies  = pd.read_csv("/kaggle/input/nicedataset/movies.csv")     # item_id,title,genre,avg_rating,num_ratings,description
users   = pd.read_csv("/kaggle/input/nicedataset/users.csv")      # user_id,gender,age,occupation,zip_code

# 1) Label-encode user_id, item_id
user_enc = LabelEncoder(); ratings["uid"] = user_enc.fit_transform(ratings.user_id)
item_enc = LabelEncoder(); ratings["iid"] = item_enc.fit_transform(ratings.item_id)
num_users = ratings.uid.nunique()
num_items = ratings.iid.nunique()

# 2) Encode user metadata:
#   - gender: M→1, F→0
#   - age: normalized [0,1]
#   - occupation: one-hot
users["gender_bin"] = (users.gender == "M").astype(int)
max_age = users.age.max()
users["age_norm"] = users.age / max_age
occ_ohe = pd.get_dummies(users.occupation, prefix="occ")
users = pd.concat([users, occ_ohe], axis=1)
# Build a matrix [num_users × meta_dim]
user_meta = users.sort_values("user_id").reset_index(drop=True)
meta_cols = ["gender_bin", "age_norm"] + list(occ_ohe.columns)
user_meta_matrix = torch.tensor(
    user_meta[meta_cols].values.astype(np.float32),
    device=device
)  # shape: [num_users, meta_dim]

print(f"Users={num_users}, Items={num_items}, meta_dim={user_meta_matrix.shape[1]}")

# 3) Train/test split theo timestamp (80/20)
ratings = ratings.sort_values("timestamp")
train_df, test_df = train_test_split(ratings, test_size=0.2, shuffle=False)
print(f"Train interactions={len(train_df)}, Test interactions={len(test_df)}")

# 4) Chuẩn bị text để encode LLM
movies["text_input"] = (
    movies.title.fillna("") + " " +
    movies.genre.fillna("") + " " +
    movies.description.fillna("")
)


Users=6040, Items=3706, meta_dim=23
Train interactions=800167, Test interactions=200042


In [4]:
# === Cell Save 2b: Lưu tạm dữ liệu đã tiền xử lý (sau Cell 2) ===
import pandas as pd
import torch
import gc

# Giả định các biến từ Cell 2: train_df, test_df, movies, users, user_meta_matrix
train_df.to_parquet("train_df.parquet")
test_df.to_parquet("test_df.parquet")
movies.to_parquet("movies.parquet")
users.to_parquet("users.parquet")
torch.save(user_meta_matrix.cpu(), "user_meta_matrix.pt")

# Giải phóng bộ nhớ những biến lớn
del train_df, test_df, movies, users, user_meta_matrix
gc.collect()


374

In [5]:
# === Cell Load 3a: Tải lại dữ liệu trước khi sinh LLM embedding (Cell 3) ===
import pandas as pd
import torch

# Load lại DataFrame và tensor metadata
train_df = pd.read_parquet("train_df.parquet")
test_df  = pd.read_parquet("test_df.parquet")
movies   = pd.read_parquet("movies.parquet")
users    = pd.read_parquet("users.parquet")

# Load lại user metadata matrix
user_meta_matrix = torch.load("user_meta_matrix.pt", map_location=device)


In [6]:


# 🛠 Đồng bộ thứ tự item trong movies với item_enc.classes_
# item_enc.classes_ chứa item_id (gốc) đã được mã hóa thành 0...num_items-1
used_item_ids = item_enc.classes_

# Sắp xếp lại movies theo thứ tự item_id đã mã hóa
movies = movies.set_index("item_id").loc[used_item_ids].reset_index()
assert len(movies) == num_items  # đảm bảo đúng kích thước

# Chuẩn bị text input để encode
item_texts = (
    movies["title"].fillna("") + " " +
    movies["genre"].fillna("") + " " +
    movies["description"].fillna("")
).tolist()

# Encode bằng LLM
llm = SentenceTransformer(cfg.llm_model, device=device)
item_llm_emb = llm.encode(
    item_texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    device=device
)

# Kiểm tra và convert sang tensor
assert item_llm_emb.shape == (num_items, cfg.llm_dim), "Số lượng hoặc chiều embedding không khớp"
item_llm_emb = torch.tensor(item_llm_emb, dtype=torch.float32, device=device)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/58 [00:00<?, ?it/s]

In [7]:
# === Cell Save 3b: Lưu tạm LLM embeddings (sau Cell 3) ===
import torch, gc

# Giả định biến item_llm_emb đã nằm trên GPU hoặc CPU
torch.save(item_llm_emb.cpu(), "item_llm_emb.pt")

# Giải phóng bộ nhớ
del item_llm_emb
gc.collect()


82

In [8]:
# === Cell Load 4a: Tải lại LLM embedding trước khi dùng trong Dataset (Cell 4) ===
import torch

# Load lại embedding vào đúng device
item_llm_emb = torch.load("item_llm_emb.pt", map_location=device)


In [9]:
# Cell 4: Dataset with negative sampling + metadata + llm_emb
# class HybridDataset(Dataset):
#     def __init__(self, df, num_items, user_meta, llm_emb, num_neg=4):
#         self.df        = df.reset_index(drop=True)
#         self.num_items = num_items
#         self.user_meta = user_meta
#         self.llm_emb   = llm_emb
#         self.num_neg   = num_neg
#         self.pos_set   = set(zip(df.uid, df.iid))
#         self.users, self.items, self.labels = self._prepare()

#     def _prepare(self):
#         U,I,L = [],[],[]
#         for u,i in zip(self.df.uid, self.df.iid):
#             U.append(u); I.append(i); L.append(1.0)
#             for _ in range(self.num_neg):
#                 neg = np.random.randint(self.num_items)
#                 while (u, neg) in self.pos_set:
#                     neg = np.random.randint(self.num_items)
#                 U.append(u); I.append(neg); L.append(0.0)
#         return U,I,L

#     def __len__(self): return len(self.labels)

#     def __getitem__(self, idx):
#         u = torch.LongTensor([self.users[idx]])
#         i = torch.LongTensor([self.items[idx]])
#         l = torch.FloatTensor([self.labels[idx]])
#         meta = self.user_meta[self.users[idx]].unsqueeze(0)  # [1,meta_dim]
#         ll   = self.llm_emb[self.items[idx]].unsqueeze(0)   # [1,llm_dim]
#         return u, i, meta, ll, l

# train_ds = HybridDataset(train_df, num_items, user_meta_matrix, item_llm_emb, cfg.num_neg)
# test_ds  = HybridDataset(test_df,  num_items, user_meta_matrix, item_llm_emb, 0)

# train_loader = DataLoader(train_ds,
#                           batch_size=cfg.batch_size,
#                           shuffle=True,  num_workers=4)
# test_loader  = DataLoader(test_ds,
#                           batch_size=cfg.batch_size,
#                           shuffle=False, num_workers=4)
# === Cell 4: Dataset dùng rating thật (explicit feedback) ===
# === Cell 4: Dataset dùng rating thật (explicit feedback) ===
class RatingDataset(Dataset):
    def __init__(self, df, user_meta, item_llm_emb):
        self.df = df.reset_index(drop=True)
        self.users = df["uid"].tolist()
        self.items = df["iid"].tolist()
        self.ratings = df["rating"].tolist()
        self.user_meta = user_meta.cpu()       # để tránh lỗi multiprocess
        self.item_llm = item_llm_emb.cpu()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        u = torch.LongTensor([self.users[idx]])
        i = torch.LongTensor([self.items[idx]])
        r = torch.FloatTensor([self.ratings[idx]])   # dùng rating thật

        meta = self.user_meta[self.users[idx]].unsqueeze(0)
        ll   = self.item_llm[self.items[idx]].unsqueeze(0)
        return u, i, meta, ll, r

# Tạo Dataset + DataLoader không cần negative sampling
train_ds = RatingDataset(train_df, user_meta_matrix, item_llm_emb)
test_ds  = RatingDataset(test_df,  user_meta_matrix, item_llm_emb)

train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True,  num_workers=0)
test_loader  = DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=0)



In [10]:
# === Cell 5: Mô hình NeuMF + Metadata + LLM cho Rating Prediction ===
class HybridNeuMF(nn.Module):
    def __init__(self, n_users, n_items, mf_dim, mlp_layers, meta_dim, llm_dim):
        super().__init__()
        self.user_mf = nn.Embedding(n_users, mf_dim)
        self.item_mf = nn.Embedding(n_items, mf_dim)

        mlp_input = mlp_layers[0]
        self.user_mlp = nn.Embedding(n_users, mlp_input // 2)
        self.item_mlp = nn.Embedding(n_items, mlp_input // 2)

        mlp_blocks = []
        for in_d, out_d in zip(mlp_layers[:-1], mlp_layers[1:]):
            mlp_blocks += [nn.Dropout(0.2), nn.Linear(in_d, out_d), nn.ReLU()]
        self.mlp = nn.Sequential(*mlp_blocks)

        fusion_dim = mf_dim + mlp_layers[-1] + meta_dim + llm_dim
        self.predict = nn.Sequential(
            nn.Linear(fusion_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)      # ❌ không dùng sigmoid
        )

    def forward(self, u, i, meta, ll):
        mu = self.user_mf(u).squeeze(1)
        mi = self.item_mf(i).squeeze(1)
        mf_vec = mu * mi

        xu = self.user_mlp(u).squeeze(1)
        xi = self.item_mlp(i).squeeze(1)
        mlp_vec = self.mlp(torch.cat([xu, xi], dim=1))

        x = torch.cat([mf_vec, mlp_vec, meta.squeeze(1), ll.squeeze(1)], dim=1)
        return self.predict(x)

# Khởi tạo model và loss
model = HybridNeuMF(
    num_users, num_items,
    mf_dim=cfg.mf_dim,
    mlp_layers=cfg.mlp_layers,
    meta_dim=user_meta_matrix.shape[1],
    llm_dim=cfg.llm_dim
).to(device)

criterion = nn.MSELoss()   # ✅ Dùng MSELoss để dự đoán rating
optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)

wandb.watch(model, log="all", log_freq=100)


In [11]:
import math
import numpy as np
import torch

# --- Top-K Ranking Metrics ---
def precision_at_k(ranked, truth, k):
    return len(set(ranked[:k]) & set(truth)) / k

def recall_at_k(ranked, truth, k):
    return len(set(ranked[:k]) & set(truth)) / len(truth) if len(truth) > 0 else 0.0

def ndcg_at_k(ranked, truth, k):
    dcg = sum(1 / math.log2(idx + 2) for idx, item in enumerate(ranked[:k]) if item in truth)
    idcg = sum(1 / math.log2(i + 2) for i in range(min(len(truth), k)))
    return dcg / idcg if idcg > 0 else 0.0

def map_at_k(ranked, truth, k):
    hits, s = 0, 0.0
    for idx, item in enumerate(ranked[:k]):
        if item in truth:
            hits += 1
            s += hits / (idx + 1)
    return s / len(truth) if len(truth) > 0 else 0.0

def mrr_at_k(ranked, truth, k):
    for idx, item in enumerate(ranked[:k]):
        if item in truth:
            return 1 / (idx + 1)
    return 0.0

# --- RMSE cho rating thật (explicit feedback) ---
def rmse_real(model, loader):
    model.eval()
    se, n = 0.0, 0
    with torch.no_grad():
        for u, i, meta, ll, r in loader:
            u, i = u.to(device), i.to(device)
            meta, ll, r = meta.to(device), ll.to(device), r.to(device)
            pred = model(u, i, meta, ll).view(-1)
            r = r.view(-1)
            se += ((pred - r) ** 2).sum().item()
            n += r.size(0)
    return math.sqrt(se / n)

# --- Hàm đánh giá toàn diện ---
@torch.no_grad()
def evaluate_full(model, train_df, test_df, K, test_loader):
    model.eval()

    # Tập item đã thấy (để mask)
    train_map = train_df.groupby("uid")["iid"].apply(set).to_dict()
    test_map  = test_df.groupby("uid")["iid"].apply(list).to_dict()

    P_list, R_list, N_list, MAP_list, MRR_list = [], [], [], [], []

    for u, truth in test_map.items():
        if len(truth) == 0:
            continue  # bỏ qua user không có ground truth

        users = torch.LongTensor([u] * num_items).to(device)
        items = torch.arange(num_items).to(device)
        metas = user_meta_matrix[u].unsqueeze(0).repeat(num_items, 1).to(device)
        lls   = item_llm_emb.to(device)

        scores = model(users, items, metas.unsqueeze(1), lls.unsqueeze(1)).view(-1).cpu().numpy()

        # Mask item đã từng thấy trong train
        for it in train_map.get(u, []):
            scores[it] = -np.inf

        ranked = np.argsort(-scores)

        P_list.append(precision_at_k(ranked, truth, K))
        R_list.append(recall_at_k(ranked, truth, K))
        N_list.append(ndcg_at_k(ranked, truth, K))
        MAP_list.append(map_at_k(ranked, truth, K))
        MRR_list.append(mrr_at_k(ranked, truth, K))

    return {
        "Precision@K": np.mean(P_list),
        "Recall@K":    np.mean(R_list),
        "NDCG@K":      np.mean(N_list),
        "MAP@K":       np.mean(MAP_list),
        "MRR@K":       np.mean(MRR_list),
        "RMSE":        rmse_real(model, test_loader)
    }


In [12]:
results = []
for ep in range(1, cfg.epochs + 1):
    model.train()
    total_loss = 0.0

    for u, i, meta, ll, r in tqdm(train_loader, desc=f"Epoch {ep}"):
        u, i = u.to(device), i.to(device)
        meta, ll, r = meta.to(device), ll.to(device), r.to(device)

        optimizer.zero_grad()
        pred = model(u, i, meta, ll).view(-1)
        loss = criterion(pred, r.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * r.size(0)

    train_loss = total_loss / len(train_ds)
    metrics = evaluate_full(model, train_df, test_df, cfg.K, test_loader)
    metrics["Train Loss"] = train_loss
    metrics["epoch"] = ep

    wandb.log(metrics)
    results.append(metrics)

    # 🖨️ In toàn bộ các metric (trừ epoch)
    print(f"Epoch {ep} — " + "  ".join(f"{k}={v:.4f}" for k, v in metrics.items() if k != "epoch"))


Epoch 1:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1 — Precision@K=0.0402  Recall@K=0.0045  NDCG@K=0.0379  MAP@K=0.0013  MRR@K=0.0858  RMSE=1.0303  Train Loss=1.6409


Epoch 2:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 2 — Precision@K=0.0875  Recall@K=0.0104  NDCG@K=0.0826  MAP@K=0.0036  MRR@K=0.1464  RMSE=0.9961  Train Loss=1.0235


Epoch 3:   0%|          | 0/782 [00:00<?, ?it/s]

...

Epoch 8 — Precision@K=0.1464  Recall@K=0.0205  NDCG@K=0.1519  MAP@K=0.0092  MRR@K=0.2837  RMSE=0.9797  Train Loss=0.6858


Epoch 9:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 9 — Precision@K=0.1384  Recall@K=0.0186  NDCG@K=0.1384  MAP@K=0.0078  MRR@K=0.2492  RMSE=0.9799  Train Loss=0.6262


Epoch 10:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 10 — Precision@K=0.1514  Recall@K=0.0205  NDCG@K=0.1683  MAP@K=0.0095  MRR@K=0.3571  RMSE=0.9711  Train Loss=0.5863


In [14]:
# ==== Cell: Inspect one batch ====
batch = next(iter(train_loader))
print(f"Batch length: {len(batch)}")
for i, t in enumerate(batch):
    try:
        print(f"  idx {i}: shape={tuple(t.shape)}, dtype={t.dtype}")
    except AttributeError:          # chẳng hạn nếu phần tử là int/float
        print(f"  idx {i}: value={t}")


Batch length: 5
  idx 0: shape=(1024, 1), dtype=torch.int64
  idx 1: shape=(1024, 1), dtype=torch.int64
  idx 2: shape=(1024, 1, 23), dtype=torch.float32
  idx 3: shape=(1024, 1, 384), dtype=torch.float32
  idx 4: shape=(1024, 1), dtype=torch.float32


In [16]:
# ==== Cell: Compute HR@K with metadata & LLM input ====
K = 10
n_neg = 100
NUM_ITEMS = num_items  # tổng số item sau khi encode

def compute_hit_rate(model, data_loader, K=10, n_neg=100,
                     device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Tính HR@K cho model HybridNeuMF(user, item, meta, ll)
    Dựa trên batch có 5 trường:
      [0] user_id (B,1), [1] pos_item (B,1), [2] user_meta (B,1,23), [3] item_llm (B,1,384), [4] rating
    """
    model.eval()
    hits, total = 0, 0

    with torch.no_grad():
        for batch in data_loader:
            user      = batch[0].squeeze(1).to(device)     # (B,)
            pos_item  = batch[1].squeeze(1).to(device)     # (B,)
            user_meta = batch[2].squeeze(1).to(device)     # (B, 23)
            item_llm  = batch[3].squeeze(1).to(device)     # (B, 384)

            B = user.size(0)

            # Sample negatives
            neg_items = torch.randint(
                0, NUM_ITEMS, size=(B, n_neg), device=device
            )
            dup_mask = (neg_items == pos_item.unsqueeze(1))
            while dup_mask.any():
                neg_items[dup_mask] = torch.randint(
                    0, NUM_ITEMS, size=(dup_mask.sum(),), device=device
                )
                dup_mask = (neg_items == pos_item.unsqueeze(1))

            # Candidates = [pos_item] + neg_items
            candidates = torch.cat(
                [pos_item.unsqueeze(1), neg_items], dim=1
            )  # (B, n_neg + 1)

            users_flat = user.unsqueeze(1).expand_as(candidates).reshape(-1)
            items_flat = candidates.reshape(-1)

            # Mở rộng user_meta & item_llm tương ứng (broadcast đúng chiều)
            meta_expanded = user_meta.unsqueeze(1).expand(B, n_neg + 1, -1).reshape(-1, user_meta.shape[-1])
            llm_expanded  = item_llm.unsqueeze(1).expand(B, n_neg + 1, -1).reshape(-1, item_llm.shape[-1])

            # Gọi forward
            scores = model(users_flat, items_flat, meta_expanded, llm_expanded)
            scores = scores.reshape(candidates.shape)

            _, topk_idx = scores.topk(K, dim=1)
            hits += (topk_idx == 0).any(dim=1).sum().item()
            total += B

    return hits / total if total else 0.0

# 🔍 Đánh giá HR@10
hrK_train = compute_hit_rate(model, train_loader, K=K, n_neg=n_neg)
hrK_test  = compute_hit_rate(model, test_loader,  K=K, n_neg=n_neg)

print(f"✅ HR@{K} on train set: {hrK_train:.4f}")
print(f"✅ HR@{K} on test  set: {hrK_test :.4f}")


✅ HR@10 on train set: 0.3159
✅ HR@10 on test  set: 0.2743


In [ ]:
# Cell 9: Show & save
df_res = pd.DataFrame(results).set_index("epoch")
display(df_res)
torch.save(model.state_dict(), "/kaggle/working/hybrid_neumf_llm.pth")
df_res.to_csv("/kaggle/working/hybrid_neumf_llm_metrics.csv")
print("✅ Saved model & metrics.")
